# Installs

In [1]:
!pip3 install openpyxl -qq

In [22]:

import pandas as pd
import jsonlines
import unicodedata
from difflib import SequenceMatcher
import re
import os

# Read in Gold Standard

In [3]:
tbl = pd.read_excel('C:/Users/natha/OneDrive/Desktop/format_converter/bsb_tables.xlsx', sheet_name='biblosinterlinear96')
tbl = tbl.set_axis(tbl.iloc[0], axis=1)
tbl = tbl[1:]
tbl.head()

In [6]:
data = tbl[['Vs', 'WLC / Nestle Base {TR} ⧼RP⧽ (WH) 〈NE〉 [NA] ‹SBL› [[ECM]]', 'Verse', 'BSB Version']]
data.head(n=12)

,Vs,WLC / Nestle Base {TR} ⧼RP⧽ (WH) 〈NE〉 [NA] ‹SBL› [[ECM]],Verse,BSB Version
1,1,בְּרֵאשִׁ֖ית,Genesis 1:1,In the beginning
2,1,אֱלֹהִ֑ים,NaN,God
3,1,אֵ֥ת,NaN,-
4,1,בָּרָ֣א,NaN,created
5,1,הַשָּׁמַ֖יִם,NaN,the heavens
6,1,וְאֵ֥ת,NaN,and
7,1,הָאָֽרֶץ׃,NaN,the earth .
8,1,NaN,NaN,NaN
9,1,NaN,NaN,NaN
10,1,NaN,NaN,NaN


In [7]:
verse_mapping = {}

for index, row in tbl.iterrows():
    if not pd.isna(row['Verse']):
        verse_num = row['Vs']
        verse_text = row['Verse']
        verse_mapping[verse_num] = verse_text

In [9]:
for index, row in data.iterrows():
    verse_num = row['Vs']
    if verse_num in verse_mapping:
        row['Verse'] = verse_mapping[verse_num]

In [10]:
data.head(n=12)

,Vs,WLC / Nestle Base {TR} ⧼RP⧽ (WH) 〈NE〉 [NA] ‹SBL› [[ECM]],Verse,BSB Version
1,1,בְּרֵאשִׁ֖ית,Genesis 1:1,In the beginning
2,1,אֱלֹהִ֑ים,Genesis 1:1,God
3,1,אֵ֥ת,Genesis 1:1,-
4,1,בָּרָ֣א,Genesis 1:1,created
5,1,הַשָּׁמַ֖יִם,Genesis 1:1,the heavens
6,1,וְאֵ֥ת,Genesis 1:1,and
7,1,הָאָֽרֶץ׃,Genesis 1:1,the earth .
8,1,NaN,Genesis 1:1,NaN
9,1,NaN,Genesis 1:1,NaN
10,1,NaN,Genesis 1:1,NaN


In [11]:
book_codes = {
    "Genesis": "GEN", "Exodus": "EXO", "Leviticus": "LEV", "Numbers": "NUM", "Deuteronomy": "DEU", "Joshua": "JOS",
    "Judges": "JDG", "Ruth": "RUT", "1 Samuel": "1SA", "2 Samuel": "2SA", "1 Kings": "1KI", "2 Kings": "2KI",
    "1 Chronicles": "1CH", "2 Chronicles": "2CH", "Ezra": "EZR", "Nehemiah": "NEH", "Esther": "EST", "Job": "JOB",
    "Psalms": "PSA", "Proverbs": "PRO", "Ecclesiastes": "ECC", "Song of Solomon": "SNG", "Isaiah": "ISA", "Jeremiah": "JER",
    "Lamentations": "LAM", "Ezekiel": "EZK", "Daniel": "DAN", "Hosea": "HOS", "Joel": "JOL", "Amos": "AMO",
    "Obadiah": "OBA", "Jonah": "JON", "Micah": "MIC", "Nahum": "NAM", "Habakkuk": "HAB", "Zephaniah": "ZEP",
    "Haggai": "HAG", "Zechariah": "ZEC", "Malachi": "MAL", "Matthew": "MAT", "Mark": "MRK", "Luke": "LUK",
    "John": "JHN", "Acts": "ACT", "Romans": "ROM", "1 Corinthians": "1CO", "2 Corinthians": "2CO", "Galatians": "GAL",
    "Ephesians": "EPH", "Philippians": "PHP", "Colossians": "COL", "1 Thessalonians": "1TH", "2 Thessalonians": "2TH", "1 Timothy": "1TI",
    "2 Timothy": "2TI", "Titus": "TIT", "Philemon": "PHM", "Hebrews": "HEB", "James": "JAS", "1 Peter": "1PE",
    "2 Peter": "2PE", "1 John": "1JN", "2 John": "2JN", "3 John": "3JN", "Jude": "JUD", "Revelation": "REV"
}

In [13]:
for index, row in data.iterrows():
    if not pd.isna(row['Verse']):
        verse_num = row['Vs']
        if verse_num in verse_mapping:
            book_title = row['Verse'].rsplit(' ', 1)[0]
            if book_title in book_codes:
                book_title = book_codes[book_title]

                row['Verse'] = row['Verse'].replace(row['Verse'].rsplit(' ', 1)[0], book_title)

In [14]:
data.head(n=12)

,Vs,WLC / Nestle Base {TR} ⧼RP⧽ (WH) 〈NE〉 [NA] ‹SBL› [[ECM]],Verse,BSB Version
1,1,בְּרֵאשִׁ֖ית,GEN 1:1,In the beginning
2,1,אֱלֹהִ֑ים,GEN 1:1,God
3,1,אֵ֥ת,GEN 1:1,-
4,1,בָּרָ֣א,GEN 1:1,created
5,1,הַשָּׁמַ֖יִם,GEN 1:1,the heavens
6,1,וְאֵ֥ת,GEN 1:1,and
7,1,הָאָֽרֶץ׃,GEN 1:1,the earth .
8,1,NaN,GEN 1:1,NaN
9,1,NaN,GEN 1:1,NaN
10,1,NaN,GEN 1:1,NaN


In [15]:
clean_data = data.dropna()

In [16]:
clean_data = clean_data.rename(columns={'WLC / Nestle Base {TR} ⧼RP⧽ (WH) 〈NE〉 [NA] ‹SBL› [[ECM]]': 'Macula'})
clean_data = clean_data[['Verse','Macula', 'BSB Version']]
clean_data.head(n=12)

,Verse,Macula,BSB Version
1,GEN 1:1,בְּרֵאשִׁ֖ית,In the beginning
2,GEN 1:1,אֱלֹהִ֑ים,God
3,GEN 1:1,אֵ֥ת,-
4,GEN 1:1,בָּרָ֣א,created
5,GEN 1:1,הַשָּׁמַ֖יִם,the heavens
6,GEN 1:1,וְאֵ֥ת,and
7,GEN 1:1,הָאָֽרֶץ׃,the earth .
11,GEN 1:2,וְהָאָ֗רֶץ,Now the earth
12,GEN 1:2,הָיְתָ֥ה,was
13,GEN 1:2,תֹ֙הוּ֙,formless


# Read in Alignments

In [17]:
alignments = 'data-chunk-aa.jsonl'

with jsonlines.open(alignments) as file:
    alignment_objs = [alignment for alignment in file]

# Compare

In [18]:
def get_alignments(alignments, verse_table):
    similarity_scores = []
    for alignment in alignments:
        align_eng = alignment['English phrase']['original-text-value']
        mac_words = alignment['Macula phrase']['original-text-value'].split(' ')
        gold_eng = []
        for word in mac_words:
            word = unicodedata.normalize('NFKC', word)
            word =  re.sub(r'[^\w\s]', '', word)
            matching_rows = verse_table.loc[verse_table['Macula'] == word]
            if not matching_rows.empty:
                # Get the corresponding string from the 'BSB Version' column of the first row
                bsb_version = matching_rows.iloc[0]['BSB Version']
                gold_eng.append(bsb_version)

        joined_string = " ".join(gold_eng)
        gold_phrase = re.sub(r'\s+', ' ', joined_string)
        
        similarity_ratio = SequenceMatcher(None, align_eng, gold_phrase).ratio()
        similarity_percentage = similarity_ratio * 100
        similarity_percentage = round(similarity_percentage, 4)
        similarity_scores.append(similarity_percentage)
        
    return similarity_scores

In [19]:
def find_similarity(alignment_obj):
    current_verse = clean_data[clean_data['Verse'] == alignment_obj['vref']]
    current_verse['Macula'] = current_verse['Macula'].apply(lambda x: unicodedata.normalize('NFKC', x))
    current_verse['BSB Version'] = current_verse['BSB Version'].str.replace(r'[^\w\s]', '')
    current_verse = current_verse.reset_index(drop=True)
    scores = get_alignments(alignment_obj['alignments'], current_verse)
    return scores

In [20]:
all_scores = []

for i in alignment_objs:
    scores = find_similarity(i)
    all_scores += scores
    
average = sum(all_scores) / len(all_scores)
print(round(average, 4))

C:\Users\natha\AppData\Local\Temp\ipykernel_23788\1344423905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_verse['Macula'] = current_verse['Macula'].apply(lambda x: unicodedata.normalize('NFKC', x))
C:\Users\natha\AppData\Local\Temp\ipykernel_23788\1344423905.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  current_verse['BSB Version'] = current_verse['BSB Version'].str.replace(r'[^\w\s]', '')
C:\Users\natha\AppData\Local\Temp\ipykernel_23788\1344423905.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

75.338


In [26]:
folder_path = 'C:/Users/natha/OneDrive/Desktop/format_converter/data'

for filename in os.listdir(folder_path):
    # Check if the item is a file
    if os.path.isfile(os.path.join(folder_path, filename)):
        # Perform operations on the file
        file_path = os.path.join(folder_path, filename)
        with jsonlines.open(file_path) as file:
            alignment_objs = [alignment for alignment in file]
            
        all_scores = []

        for i in alignment_objs:
            try:
                scores = find_similarity(i)
                all_scores += scores
            except:
                pass
            
        average = sum(all_scores) / len(all_scores)
        print(filename)
        print(round(average, 4))

C:\Users\natha\AppData\Local\Temp\ipykernel_23788\1344423905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_verse['Macula'] = current_verse['Macula'].apply(lambda x: unicodedata.normalize('NFKC', x))
C:\Users\natha\AppData\Local\Temp\ipykernel_23788\1344423905.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  current_verse['BSB Version'] = current_verse['BSB Version'].str.replace(r'[^\w\s]', '')
C:\Users\natha\AppData\Local\Temp\ipykernel_23788\1344423905.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

fr-data-chunk-aa.jsonl
75.338
fr-data-chunk-ab.jsonl
7.3256
fr-data-chunk-ac.jsonl
56.4636
sp-data-chunk-aa.jsonl
8.755
sp-data-chunk-ab.jsonl
10.8429
sp-data-chunk-ac.jsonl
64.4392
tp-data-chunk-aa.jsonl
75.338
tp-data-chunk-ab.jsonl
0.7359
tp-data-chunk-ac.jsonl
0.9285
